In [7]:
import polars as pl
import pandas as pd
import numpy as np
import os
import math
import shutil
import sys

Se cargan los datasets a dataframes, para checar a groso modo la estructura de los mismos, se pasan a un dataframe y se guardan en archivos parquets particionados para trabajar desde github con todo el equipo

In [11]:
dfreviews=pd.read_json(r'Yelp\review-002.json',lines=True,engine='pyarrow',dtype_backend='pyarrow')
dfreviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [9]:
dfreviews.to_parquet(r'parquets\reviews_yelp.parquet')

In [13]:
dfbusiness=pd.read_pickle(r'Yelp\business.pkl')
dfbusiness=dfbusiness.iloc[:,0:14]
dfbusiness.to_parquet(r'parquets\business.parquet')

In [ ]:
dfbusiness['categories']=dfbusiness.categories.str.split(', ')

In [ ]:
dfbusiness=dfbusiness.explode(column='categories')
dfestado_categoria=dfbusiness.groupby(['state','categories'])[['state','categories']].size().reset_index(name='count')

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,TN,63123,38.565165,-90.321087,3.0,19,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Pubs, Restaurants, Italian, Bars, American (Tr...",None
20,WKMJwqnfZKsAae75RMP6jA,Roast Coffeehouse and Wine Bar,10359 104 Street NW,Edmonton,LA,T5J 1B9,53.546045,-113.499169,4.0,40,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Coffee & Tea, Food, Cafes, Bars, Wine Bars, Re...","{'Friday': '8:0-18:0', 'Monday': '8:0-18:0', '..."
28,QdN72BWoyFypdGJhhI5r7g,Bar One,767 S 9th St,Philadelphia,PA,19147,39.939825,-75.157447,4.0,65,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Cocktail Bars, Bars, Italian, Nightlife, Resta...","{'Friday': '12:0-2:0', 'Monday': '16:0-0:0', '..."
39,fSCNwMtNNQY9QT69Cj9fiA,Sierra Pro Events,,Sparks,PA,89431,39.540154,-119.748395,5.0,7,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Musicians, DJs, Karaoke, Event Planning & Serv...","{'Friday': '9:0-17:0', 'Monday': '9:0-17:0', '..."
46,JX4tUpd09YFchLBuI43lGw,Naked Cyber Cafe & Espresso Bar,10303 108 Street NW,Edmonton,PA,T5J 1L7,53.544682,-113.506589,4.0,12,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Arts & Entertainment, Music Venues, Internet S...","{'Friday': '11:0-1:0', 'Monday': '11:0-1:0', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150274,9iCum5W48TqNVDTZCJUKjg,Plantation Pub,8321 Sawyer Brown Rd,Bellevue,FL,37221,36.070748,-86.947891,3.5,40,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Sports Bars, Nightlife, Pubs, Bars","{'Friday': '11:0-1:0', 'Monday': '11:0-0:0', '..."
150275,IeSD0nMKRFYUTnR5nZH1CQ,HighWire Lounge,14 S Arizona Ave,Tucson,CA,85701,32.221828,-110.967969,3.5,111,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Bars, Tapas Bars, Restaurants, Nightlife, Gast...","{'Friday': '14:0-2:0', 'Monday': None, 'Saturd..."
150291,-R9t30RvsG-Rtiq63yuEmg,Crescent No Lawyers Bar-Grill,5500 W Franklin Rd,Boise,IN,83706,43.603368,-116.250063,2.5,55,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","American (Traditional), Nightlife, Bars, Resta...","{'Friday': '11:0-2:0', 'Monday': '11:0-2:0', '..."
150292,esBGrrmuZzSiECyRBoKvvA,Colony Grill - St. Petersburg,670 Central Ave,St. Petersburg,FL,33701,27.770872,-82.643069,4.5,38,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Bars, Beer Bar, Nightlife, Wine Bars, Pizza, R...","{'Friday': '11:30-0:30', 'Monday': '11:30-23:0..."


In [56]:
dfestado_categoria[dfestado_categoria['categories'].str.contains('Nightlife')]

,state,categories,count
13,AB,"Active Life, Arts & Crafts, Nightlife, Shoppin...",1
16,AB,"Active Life, Bars, Nightlife, Bowling, Venues ...",1
51,AB,"Active Life, Nightlife, Sports Bars, Dog Parks...",1
85,AB,"Adult Entertainment, Axe Throwing, Nightlife, ...",1
86,AB,"Adult Entertainment, Nightlife",1
...,...,...,...
106488,TN,"Wine Tours, Arts & Entertainment, Nightlife, F...",1
106491,TN,"Wineries, Arts & Entertainment, Tapas/Small Pl...",1
106493,TN,"Wineries, Food, Restaurants, Bars, Nightlife, ...",1
106495,TN,"Wineries, Wine Tasting Room, Wine Bars, Food, ...",1


In [7]:
dftip=pd.read_json(r'Yelp\tip.json',lines=True)
dftip.to_parquet(r'parquets\tip.parquet')

In [2]:
dfusers=pl.read_parquet(r'Yelp\user-001.parquet')
dfusers=dfusers.to_pandas()
dfusers.to_parquet(r'parquets\users.parquet')

In [4]:
dfusers=pd.read_parquet('parquets/users.parquet')

In [16]:
dfcheckin=pd.read_json('Yelp\checkin.json',lines=True)
dfcheckin.to_parquet(r'parquets\checkin.parquet')

In [3]:
dfbusiness.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null     

In [29]:
dftip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908915 entries, 0 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   user_id           908915 non-null  object        
 1   business_id       908915 non-null  object        
 2   text              908915 non-null  object        
 3   date              908915 non-null  datetime64[ns]
 4   compliment_count  908915 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 34.7+ MB


In [31]:
dftip.isna().sum()

user_id             0
business_id         0
text                0
date                0
compliment_count    0
dtype: int64

In [36]:
dfcheckin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131930 entries, 0 to 131929
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  131930 non-null  object
 1   date         131930 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


In [14]:
(dfusers['elite']=='').sum()/len(dfusers)*100

94.45473184089833

In [ ]:
dfusers.info()

# Partición para github

In [82]:
os.makedirs('FinalProject_YelpGoogleMaps\\datasets\\parquets')

In [83]:
for a in os.listdir('parquets'):
    if a =='users.parquet':
        continue
    tamaño_archivo=round((os.path.getsize('parquets\\'+a))/1024,2)
    if tamaño_archivo>100000:
        particiones=math.ceil(tamaño_archivo/100000)+2
        df=pd.read_parquet('parquets\\'+a)
        particion=np.array_split(df,particiones)
        for n,d in enumerate(particion):
            d.to_parquet('FinalProject_YelpGoogleMaps\\datasets\\parquets\\'+a.split('.')[0]+str(n+1).zfill(2)+'.parquet',index=False)
    else:
        if a.split('.')[-1]=='parquet':
            shutil.copy('parquets\\'+a,'FinalProject_YelpGoogleMaps\\datasets\\parquets\\'+a)
        

In [84]:
os.mkdir(path='FinalProject_YelpGoogleMaps\\datasets\\parquets\\reviews-Google')

In [87]:
for a in os.listdir('parquets\\reviews-Google'):
    tamaño_archivo=round((os.path.getsize('parquets\\reviews-Google\\'+a))/1024,2)
    if tamaño_archivo>100000:
        particiones=math.ceil(tamaño_archivo/100000)+1
        df=pd.read_parquet('parquets\\reviews-Google\\'+a)
        particion=np.array_split(df,particiones)
        for n,d in enumerate(particion):
            d.to_parquet('FinalProject_YelpGoogleMaps\\datasets\\parquets\\reviews-Google\\'+a.split('.')[0]+str(n+1).zfill(2)+'.parquet',index=False)
    else:
        shutil.copy('parquets\\reviews-Google\\'+a,'FinalProject_YelpGoogleMaps\\datasets\\parquets\\reviews-Google\\'+a)

In [88]:
dfusers = pd.read_parquet(r'parquets\users.parquet')

In [89]:
particion=np.array_split(dfusers,30)
def particionar(df):
        global contador
        global pesodf
        peso=(sys.getsizeof(df)/1024)/1024
        tamaño2=math.ceil(peso/100)
        particiones=np.array_split(df,tamaño2)
        for a in particiones:
             tamaño3=(sys.getsizeof(a)/1024)/1024
             if tamaño3<100:
                a.to_parquet('FinalProject_YelpGoogleMaps\\datasets\\parquets\\users'+str(contador).zfill(2)+'.parquet',index=False)
                contador+=1
                #pesodf+=(sys.getsizeof(a)/1024)/1024
             else:
                particionar(a)   
        
contador=1
pesodf=0
for b in particion:
    tamaño= sys.getsizeof(b)/1024/1024
    if tamaño>100:
        particionar(b)
    else:
        b.to_parquet('FinalProject_YelpGoogleMaps\\datasets\\parquets\\users'+str(contador).zfill(2)+'.parquet',index=False)
        contador+=1